In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
def build_base(inputSize, loss_function = "mse", optimizer = "adam"):
    model = Sequential()
    model.add(Dense(inputSize[0]//2, input_shape = inputSize, activation = "relu"))
    model.add(Dense(inputSize[0], activation = "relu"))
    model.compile(loss = loss_function, optimizer = optimizer, metrics = ["mse"])

    return model

def get_encoded_data(model, decoder_layers, xTrain, loss_function = "mse", optimizer = "adam"):
    decoder_layers.append(model.layers[-1])
    model.pop()
    model.compile(loss = loss_function, optimizer = optimizer, metrics = [loss_function])
    model.summary()
    print()
    xTrainEncoded = model.predict(xTrain)
    print(xTrainEncoded.shape)
    
    return xTrainEncoded, decoder_layers

def add_layer(model, layer_dimension):
    for layer in model.layers:
        layer.trainable = False
    model.add(Dense(layer_dimension, activation = "relu"))
    model.add(Dense(layer_dimension*2, activation = "relu"))
    model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
    model.summary()
    
    return model

# 1st Autoencoder

In [3]:
model = build_base((32*32,), "mse", "adam")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
Total params: 1,050,112
Trainable params: 1,050,112
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Treinamos a primeiro autoencoder

xTrain = np.load("../data/711_input.npy").astype("float32")

model.fit(xTrain, xTrain, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 3s 304us/sample - loss: 0.0323 - mse: 0.0323
Epoch 2/100
10499/10499 [==============================] - 1s 48us/sample - loss: 0.0299 - mse: 0.0299
Epoch 3/100
10499/10499 [==============================] - 0s 43us/sample - loss: 0.0298 - mse: 0.0298
Epoch 4/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0297 - mse: 0.02970s - loss: 0.0295 - mse
Epoch 5/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0297 - mse: 0.0297
Epoch 6/100
10499/10499 [==============================] - 0s 46us/sample - loss: 0.0297 - mse: 0.0297
Epoch 7/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0296 - mse: 0.0296
Epoch 8/100
10499/10499 [==============================] - 0s 46us/sample - loss: 0.0296 - mse: 0.0296
Epoch 9/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0296 - mse: 0.0296
Epoch 10/100
10499/10499 [

Epoch 79/100
10499/10499 [==============================] - 1s 59us/sample - loss: 0.0296 - mse: 0.0296
Epoch 80/100
10499/10499 [==============================] - 1s 75us/sample - loss: 0.0296 - mse: 0.0296
Epoch 81/100
10499/10499 [==============================] - 1s 73us/sample - loss: 0.0295 - mse: 0.0295
Epoch 82/100
10499/10499 [==============================] - 1s 63us/sample - loss: 0.0295 - mse: 0.0295
Epoch 83/100
10499/10499 [==============================] - 1s 60us/sample - loss: 0.0295 - mse: 0.0295
Epoch 84/100
10499/10499 [==============================] - 0s 43us/sample - loss: 0.0295 - mse: 0.0295
Epoch 85/100
10499/10499 [==============================] - 0s 43us/sample - loss: 0.0295 - mse: 0.0295
Epoch 86/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0295 - mse: 0.0295
Epoch 87/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0295 - mse: 0.0295
Epoch 88/100
10499/10499 [==============================] - 0s 4

# 2nd Autoencoder

In [5]:
# Retiramos a layer de decodificação, e pegamos os dados codificados para treinar o próximo autoencoder.

xTrainEncoded, decoder_layers = get_encoded_data(model, [], xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
Total params: 524,800
Trainable params: 524,800
Non-trainable params: 0
_________________________________________________________________

(10499, 512)


In [6]:
# Adicionando o segundo autoencoder

model = add_layer(model, 256)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               131584    
Total params: 787,712
Trainable params: 262,912
Non-trainable params: 524,800
_________________________________________________________________


In [7]:
# Treinando o segundo autoencoder

model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 65us/sample - loss: 0.0187 - mse: 0.0187
Epoch 2/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0145 - mse: 0.0145
Epoch 3/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0096 - mse: 0.0096
Epoch 4/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0071 - mse: 0.0071
Epoch 5/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0071 - mse: 0.0071
Epoch 6/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0071 - mse: 0.0071
Epoch 7/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0070 - mse: 0.0070
Epoch 8/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0070 - mse: 0.0070
Epoch 9/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0070 - mse: 0.0070
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 31us/sample - loss: 0.0070 - mse: 0.0070
Epoch 79/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0070 - mse: 0.0070
Epoch 80/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0070 - mse: 0.0070
Epoch 81/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0070 - mse: 0.0070
Epoch 82/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0070 - mse: 0.0070
Epoch 83/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0070 - mse: 0.0070
Epoch 84/100
10499/10499 [==============================] - 0s 39us/sample - loss: 0.0070 - mse: 0.0070
Epoch 85/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0070 - mse: 0.0070
Epoch 86/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0070 - mse: 0.0070
Epoch 87/100
10499/10499 [==============================] - 0s 31us/sample - 

# 3rd Autoencoder

In [8]:
# Training 3rd autoencoder

xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
Total params: 656,128
Trainable params: 131,328
Non-trainable params: 524,800
_________________________________________________________________

(10499, 256)


In [9]:
model = add_layer(model, 128)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               33024     
Total params: 722,048
Trainable params: 65,920
Non-trainable params: 656,128
_________________________________________________________________


In [10]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 51us/sample - loss: 0.0140 - mse: 0.0140
Epoch 2/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0061 - mse: 0.0061
Epoch 3/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0041 - mse: 0.0041
Epoch 4/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0038 - mse: 0.0038
Epoch 5/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0037 - mse: 0.0037
Epoch 6/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0037 - mse: 0.0037
Epoch 7/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0036 - mse: 0.0036
Epoch 8/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0036 - mse: 0.0036
Epoch 9/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0036 - mse: 0.0036
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 32us/sample - loss: 0.0026 - mse: 0.0026
Epoch 80/100
10499/10499 [==============================] - 0s 28us/sample - loss: 0.0026 - mse: 0.0026
Epoch 81/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 82/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 83/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 84/100
10499/10499 [==============================] - 0s 28us/sample - loss: 0.0026 - mse: 0.0026
Epoch 85/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 86/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 87/100
10499/10499 [==============================] - 0s 36us/sample - loss: 0.0026 - mse: 0.0026
Epoch 88/100
10499/10499 [==============================] - 0s 31us/sample - 

# 4th Autoencoder

In [11]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
Total params: 689,024
Trainable params: 32,896
Non-trainable params: 656,128
_________________________________________________________________

(10499, 128)


In [12]:
model = add_layer(model, 64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
Total params: 705,600
Trainable params: 16,576
Non-trainable params: 689,024
_________________________________________________________________


In [13]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 61us/sample - loss: 0.0218 - mse: 0.0218
Epoch 2/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0094 - mse: 0.00940s - loss: 0.0107 - mse: 0
Epoch 3/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0069 - mse: 0.0069
Epoch 4/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0060 - mse: 0.0060
Epoch 5/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0054 - mse: 0.0054
Epoch 6/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0052 - mse: 0.0052
Epoch 7/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0051 - mse: 0.0051
Epoch 8/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0050 - mse: 0.0050
Epoch 9/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0049 - mse: 0.0049
Epoch 10/100
10499/10499

10499/10499 [==============================] - 0s 31us/sample - loss: 0.0026 - mse: 0.0026
Epoch 80/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 81/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 82/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 83/100
10499/10499 [==============================] - 0s 29us/sample - loss: 0.0026 - mse: 0.0026
Epoch 84/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 85/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 86/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 87/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0026 - mse: 0.0026
Epoch 88/100
10499/10499 [==============================] - 0s 30us/sample - 

# 5th Autoencoder

In [14]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
Total params: 697,280
Trainable params: 8,256
Non-trainable params: 689,024
_________________________________________________________________

(10499, 64)


In [15]:
model = add_layer(model, 32)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
Total params: 701,472
Trainable params: 4,192
Non-trainable params: 697,280
______________________________________________

In [16]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 63us/sample - loss: 0.0683 - mse: 0.0683
Epoch 2/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0406 - mse: 0.0406
Epoch 3/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0393 - mse: 0.0393
Epoch 4/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0380 - mse: 0.0380
Epoch 5/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0369 - mse: 0.0369
Epoch 6/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0355 - mse: 0.0355
Epoch 7/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0348 - mse: 0.0348
Epoch 8/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0344 - mse: 0.0344
Epoch 9/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0328 - mse: 0.0328
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 31us/sample - loss: 0.0277 - mse: 0.0277
Epoch 80/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0277 - mse: 0.0277
Epoch 81/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0277 - mse: 0.0277
Epoch 82/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0277 - mse: 0.0277
Epoch 83/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0277 - mse: 0.0277
Epoch 84/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0277 - mse: 0.0277
Epoch 85/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0277 - mse: 0.0277
Epoch 86/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0277 - mse: 0.0277
Epoch 87/100
10499/10499 [==============================] - 0s 30us/sample - loss: 0.0277 - mse: 0.0277
Epoch 88/100
10499/10499 [==============================] - 0s 30us/sample - 

# Add decoder layers

In [17]:
for i in range(len(decoder_layers)-1, -1, -1):
    model.add(decoder_layers[i])
    
for layer in model.layers:
    layer.trainable = False

model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8